# TEST

In [7]:
!pwd
isLocalServer = False

%load_ext autoreload
%autoreload 2
%matplotlib inline
import json, sys, os

if isLocalServer is True:
  print(f'This is in the local server, therefore your folder path should be added to the system path.')
  
  sys_path_to_be_added = '/notebook/personal/ksuchoi216/FaceID-model/'
  if not sys_path_to_be_added in sys.path:
    sys.path.insert(0, sys_path_to_be_added)
    os.chdir(sys_path_to_be_added)

  print(f'System path as follows:')
  for path in sys.path:
    print(f'    {path}')

from utils import load_config
cfg = load_config('config_test-various-face-extraction.json')

/Volumes/ssd/code/FaceID-model
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import numpy as np
# loading numpy data

def loadNumpyImagesToDict(user_names_list, data_source_path):
  user_names = user_names_list
  data_source = data_source_path

  numpy_data = {}
  print(numpy_data)
  for i, user_name in enumerate(user_names):
    print(i, user_name)
    file_path = data_source + user_name + '_images.npy'
    numpy_data[user_name] = np.load(file_path, allow_pickle=True)

  print('example dimension: {}'.format(numpy_data['jhyoo'].shape))


In [9]:
from PIL import Image
import matplotlib.pyplot as plt

In [10]:
from deepface import DeepFace

In [11]:
# embedding data to be saved as numpy type
''' embedding numpy matrix form
      ---models--- label
      +-----------+---+
      |           | 0 |
data  |           | 0 |
      |           | 0 |
      |           | 1 |
      +-----------+---+
'''


models = cfg['models']


data_source = './data/photos_from_video/'
user_names = cfg['user_names']
print(user_names)
theNumberOfImages = cfg['theNumberOfImages']
save_folder_for_embedding_numpy = cfg['save_path_for_embedding_numpy']


emb_numpy_matrix = np.zeros(shape = (theNumerOfImages*(len(user_names)+1), len(models)+1))
print(f'emb_numpy_martix shape: {emb_numpy_matrix.shape}')

for model_num in range(len(models)):
  temp_emb_numpy_matrix = np.zeros(shape = (theNumerOfImages, len(models)+1))
  print(f'temp_emb_numpy_martix shape: {temp_emb_numpy_matrix.shape}')
  
  for label, user_name in enumerate(user_names):
    temp_emb_numpy_matrix[:, len(models)+1] = label
    
    emb_list_col = []
    for num in range(theNumberOfImages):
      img_path = data_source + user_name + '/' + str(num) + '.jpg'
      emb = DeepFace.represent(img_path=img_path, model_name = models[model_num])
      if isinstance(emb, np.ndarray):
        emb_list_col.append(emb)
    
    
    start_row = label*theNumberOfImages
    end_row = ((label+1) *  theNumberOfImages)-1
    print(f'the data row number {start_row} ~ {end_row}')
    emb_numpy_matrix[start_row:end_row, model_num] = np.array(emb_list_col)
  
  
try: 
  save_path = save_folder_for_embedding_numpy + 'embedding_numpy_matrix.npy'
  np.save(save_path_for_embedding_numpy, emb_numpy_matrix, allow_pickle=True)        
except:
  print(f"failed saving numpy data in {save_path}")

['jhyoo', 'jhoh', 'jhongyoo', 'kschoi']


ValueError: ('Confirm that opencv is installed on your environment! Expected path ', '/Users/KC/opt/anaconda3/envs/torch-cpu/lib/python3.8/site-packages/data/haarcascade_frontalface_default.xml', ' violated.')

In [ ]:
save_folder_for_embedding_numpy = cfg['save_folder_for_embedding_numpy']
load_path = save_folder_for_embedding_numpy + 'embedding_numpy_matrix.npy'
emb_numpy_matrix = np.load(load_path)

import pandas as pd
from utils import pca
import seaborn as sns
import matplotlib.pyplot as plt

models = cfg['models']
for model_num in range(len(model)):
  labels = emb_numpy_matrix[:, len(model)+1]
  emb_data = emb_numpy_matrix[:, model_num]
  
  print(f'label shape: {labels.shape} emb_data shape: {emb_data.shape}')
  pca_x1, pca_x2 = execute_pca(emb_data)
  
  combined_array = np.column_stack([pca_x1, pca_x2, labels])
  df_for_pca = pd.DataFrame(combined_array, columns = ['pca_x1', 'pca_x2', 'labels'])
  
  fig = plt.figure(1, figsize=(15, 10))
  sns.scatterplot(data = df_for_pca, x = 'pca_x1', y = 'pca_x2', hue = 'labels')
  plt.show()